  <center><img src="images/2024_reInvent_Logo_wDate_Black_V3.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

# <a name="0">AWS re:Invent 2024 | Lab 2: Detect, Measure and Remediate hallucinations  </a>
## <a name="0">Using Amazon Bedrock Agents for custom intervention when hallucinations are detected </a>

## Lab Overview

In this lab, we will set up our own custom workflow to intervene when hallucinations are detected by using [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/) and route to customer service agents bringing in humans in the loop.


##### Notebook Kernel
Please choose `Python3` as the kernel type of the top right corner of the notebook if that does not appear by default.

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This lab auto-cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#10"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>


## Use-Case Overview
We want to add our own custom intervention to the RAG powered chatbot we developed in Lab 1.
We will be using few of the RAGAS metrics like `answer correctness` and `answer similarity` to develop a custom hallucination score for measuring hallucinations. If the custom hallucination score is less than a custom threshold it indicates that the generated model response is not well aligned with the ground truth. In this situation, we notify a pool of human agents via SNS notification to assist with the query instead of providing the customer with hallucinated model response.


To set up this workflow, we leverage AWS services like Amazon Bedrock Agents, Lambdas, Amazon Knowledge Bases as shown in the architecture diagram :

The overall workflow involves the following steps as given in the diagram:
0. Data Ingestion - S3 raw PDFs ingested to Amazon Knowledge base (we covered this in Lab 1) 
1. User asks the agent a question relevant to Bedrock User Guide.
2. Agent searches for an answer inside the knowledge base.
3. The query search goes inside vector database. We are using Opensearch Serverless.
4. Relevant answer chunks are retrieved.
5. Knowledge base response is generated using `retrieve and generate` api. (covered in lab 1)
6. User question and kb response are used to invoke right action group
7. User question and kb response are passed as Lambda inputs to calculate hallucination score
8. send SNS notification if answer score is lower than the custom threshold (0.9)
9. Lambda responds with final KB response if there is no hallucination else sends response that customer agent has been asked to join shortly.
10. Final agent response shown to customer UI as elaborated in above step.




<center><img src="images/lab2-reinvent-arch-diagram-v1.png" alt="This image shows the retrieval augmented generation (RAG) system design setup with knowledge bases, S3, and AOSS. Knowledge corpus is ingested into a vector database using Amazon Bedrock Knowledge Base Agent and then RAG approach is used to work question answering. The question is converted into embeddings followed by semantic similarity search to get similar documents. With the user prompt being augmented with the RAG search response, the LLM is invoked to get the final raw response for the user." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>



#### Lab Sections

This lab notebook has the following sections:

1. <a href="#1">Environment setup and configuration</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Setup agent infrastructure</a>
4. <a href="#4">Create an agent</a>
5. <a href="#5">Associate knowledge bases, deploy agent, create alias</a>
6. <a href="#6">Invoke agent</a>
9. <a href="#7">Monitor SNS message count for Human in the Loop setup</a>
10. <a href="#8">Clean up resources</a>
11. <a href="#9">Challenge exercise and lab quiz</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.


----


Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.


## <a name="1">Environment setup and configuration</a>
(<a href="#0">Go to top</a>)

Before starting, let's import the required packages and configure the support variables:

<div style="border: 4px solid coral; text-align: left; margin: auto;">
    <p style="text-align: center; margin: auto;">If you see Throttling Error please increase `time.sleep(10)` to say 20,30 seconds.</p>
</div>


In [ ]:
%%capture
!pip install -r ../requirements.txt --quiet

In [2]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from IPython.display import Markdown
import warnings
from botocore.config import Config
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
from agent_utilities.agents_utils import *
from agent_utilities.agents_infra_utils_one_kb_setup import *

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.ERROR)
logger = logging.getLogger(__name__)

pp = pprint.PrettyPrinter(width=41, compact=True)

In [4]:
clean_up_trace_files("./trace_files/")

### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).
```
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

```

</br>

In [5]:
# getting boto3 clients for required AWS services
bedrock_boto3_config = Config(
    connect_timeout=60*10,
    read_timeout=60*10,
)

RETRY_CONFIG = Config(
    retries={
        'max_attempts': 3,            # Maximum number of retry attempts
        'mode': 'adaptive'            # Adaptive mode adjusts based on request limits
    },
    read_timeout=1000,
    connect_timeout=1000
    
)

bedrock_runtime_client = boto3.client(
    'bedrock-runtime',
    region_name=region,
    config=RETRY_CONFIG
)


sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')

bedrock_agent_client = boto3.client('bedrock-agent', config=RETRY_CONFIG)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', config=RETRY_CONFIG)
open_search_serverless_client = boto3.client('opensearchserverless', config=RETRY_CONFIG)

session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '996757723911')

In [6]:
%%capture
#test if bedrock model access has been enabled 
input_prompt = "Who was the first person to land on the sun?"
test_llm_call(input_prompt)

### <a name="3">3. Setup agent infrastructure</a>
(<a href="#0">Go to top</a>)

High level workflow:
- Setup for variables with various agent resources
- Create Lambda function for action group
- Create Knowledge Base 1 for QnA with latest the Amazon Bedrock User Guide
- Creating an agent


In [ ]:
lab1_kb_id = None
kb_list = bedrock_agent_client.list_knowledge_bases()['knowledgeBaseSummaries']
for kb in kb_list:
    if kb['name'] == 'bedrock_user_guide_kb':
        lab1_kb_id = kb['knowledgeBaseId']

print(lab1_kb_id)

In [ ]:
kb_id = None
%store -r kb_id
if kb_id is None:
    kb_id = lab1_kb_id
# if a kb already exists we can use the same, else the infra setup code will create one by itself using the bedrock user guide.
print(f"Lab 1 store kb_id :: {kb_id}")
use_existing_kb = False
existing_kb_id = None
if kb_id is not None:
    use_existing_kb = True
    existing_kb_id = kb_id
print(f"use_existing_kb :: {use_existing_kb}")
print(f"existing_kb_id :: {existing_kb_id}")

Lab 1 store kb_id :: CMKJN2QRK1
use_existing_kb :: True
existing_kb_id :: CMKJN2QRK1


In [8]:
schema_filename='hallucination_agent_openapi_schema_with_kb.json'
kb_db_file_uri='kb_hallucination'
lambda_code_uri='lambda_hallucination_detection.py'
sns_topic_name='reinvent2024_hallucination_lab2b_topic'
gt_file_name='reinvent2024-hallucinations-questions.csv'



kb_id = None
%store -r kb_id
# if a kb already exists we can use the same, else the infra setup code will create one by itself using the bedrock user guide.
print(f"Lab 1 store kb_id :: {kb_id}")
if kb_id is not None:
    use_existing_kb = True
    existing_kb_id = kb_id

print(f"use_existing_kb :: {use_existing_kb}")
print(f"existing_kb_id :: {existing_kb_id}")

Lab 1 store kb_id :: CMKJN2QRK1
use_existing_kb :: True
existing_kb_id :: CMKJN2QRK1


In [9]:
%%time
# For new KB it takes around ~6 minutes for this setup to complete on a t2.medium instance.
infra_response = setup_agent_infrastructure(schema_filename=schema_filename,
                                           kb_db_file_uri=kb_db_file_uri,
                                           lambda_code_uri=lambda_code_uri,
                                           sns_topic_name=sns_topic_name,
                                           gt_file_name=gt_file_name,
                                           use_existing_kb = use_existing_kb,
                                           existing_kb_id = existing_kb_id 
                                           )




agent_bedrock_policy :: None
agent_s3_schema_policy :: None
kb_db_bedrock_policy :: None
kb_aws_bedrock_policy :: None
kb_db_s3_policy :: None
Lambda :: LambdaAgentsHallucinationDetection is now subscribed to SNS :: arn:aws:sns:us-west-2:996757723911:reinvent2024_hallucination_lab2b_topic and subscriptionArn is arn:aws:sns:us-west-2:996757723911:reinvent2024_hallucination_lab2b_topic:84df315d-6eed-4e5b-b386-ed7df672c3e7
Using existing_kb_id :: CMKJN2QRK1
CPU times: user 138 ms, sys: 10.4 ms, total: 148 ms
Wall time: 2.59 s


In [10]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
knowledge_base_db_id = infra_response["knowledge_base_db_id"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]
kb_db_collection_name = infra_response["kb_db_collection_name"]
kb_db_bedrock_policy = infra_response["kb_db_bedrock_policy"]
kb_db_aoss_policy = infra_response["kb_db_aoss_policy"]
kb_db_s3_policy = infra_response["kb_db_s3_policy"]
agent_kb_schema_policy = infra_response["agent_kb_schema_policy"]
kb_db_role_name = infra_response["kb_db_role_name"]
kb_db_opensearch_collection_response = infra_response["kb_db_opensearch_collection_response"]

In [11]:
agent_name

'l2c24deb-reduce-hallucinations-in-genai-apps'

In [12]:
knowledge_base_db_id

'CMKJN2QRK1'

### <a name="4">Create agent</a>
(<a href="#0">Go to top</a>)


Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [13]:
# Create agent
agent_instruction = """
You are a question answering agent that helps customers answer questions from the Amazon Bedrock User Guide inside the associated knowledge base.
Next you will always use the knowledge base search result to detect and measure any hallucination using the functions provided"
"""
# anthropic.claude-3-sonnet-20240229-v1:0
# anthropic.claude-3-haiku-20240307-v1:0

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Ask questions to get answers from the latest Amazon Bedrock User Guide",
    idleSessionTTLInSeconds=3600,
    foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id

'NY5CJX56KK'

Looking at the created agent, we can see its status and agent id. We have saved the `agent_id` in a local variable to use it for the next steps

### <a name="5">Associate knowledge bases, deploy agent, create alias</a>
(<a href="#0">Go to top</a>)

After we have the agent, we still have to 
1. Create agent action group
2. Allowing agent to invoke action group Lambda
3. Associating the agent to the knowledge base
4. Prepare the agent
5. Create agent alias to deploy agent

We cover the detailed implementation inside `setup_agent_after_create()` in `agent_utilties\agents_infra_utils_one_kb_setup` python file.

Once that is done, let's use the `bedrock-agent-runtime` client to invoke this agent and ask user questions on bedrock user guide.

In [14]:
%%time
# note:: this can take around 2-3 mins 
agent_alias, agent_action_group_response = setup_agent_after_create(bedrock_agent_client, 
                                         agent_id,
                                         agent_alias_name,
                                         lambda_function,
                                         bucket_name,
                                         schema_key,
                                         lambda_name,
                                      knowledge_base_db_id,
                                        sns_topic_name)
#agent_alias_name = agent_alias['agentAlias']['agentAliasName']
agent_alias_id = agent_alias['agentAlias']['agentAliasId']
print(f"agent_alias_name :: {agent_alias_name} and agent_alias_id :: {agent_alias_id}")

agent_alias_name :: l2c24deb-workshop-alias and agent_alias_id :: MO2DZGJWTS
CPU times: user 45 ms, sys: 14.1 ms, total: 59.1 ms
Wall time: 2min 30s


### <a name="6">Invoke agent</a>
(<a href="#0">Go to top</a>)

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and loop through all user questions inside `reinvent2024-hallucinations-questions.csv` and ask them to the agent.

We set the minimum answer score threshold of at least `0.85` for the exact model response to go back to the customer as-is without bringing human in the loop.

In [15]:
# lets see the content of the user-questions and ground truth
questions_df = pd.read_csv("./reinvent2024-hallucinations-questions.csv", sep=',')
questions_df.style.set_properties(**{'text-align': 'left', 'border': '1px solid black'})
questions_df.to_string(justify='left', index=False)
with pd.option_context("display.max_colwidth", None):
    pretty_print(questions_df)

,question_id,question,ground_truth_answer
0,Q001,What models are supported by bedrock agents?,The following models are supported for use with Amazon Bedrock Agents: Anthropic Claude v2 Anthropic Claude v2.1 Anthropic Claude Instant Anthropic Claude 3 Sonnet Anthropic Claude 3 Haiku Anthropic Claude 3.5 Sonnet Titan Text G1 Premier
1,Q002,Which models can I use with Amazon Bedrock Agents?,The following models are supported for use with Amazon Bedrock Agents: Anthropic Claude v2 Anthropic Claude v2.1 Anthropic Claude Instant Anthropic Claude 3 Sonnet Anthropic Claude 3 Haiku Anthropic Claude 3.5 Sonnet Titan Text G1 Premier
2,Q003,Which are the dates for reinvent 2024?,December 2-6
3,Q004,What is the Model ID of Amazon Titan Text Premier,The Model ID of Amazon Titan Text G1 - Premier is amazon.titan-text-premier-v1:0.


In [16]:
USER_PROMPT_TEMPLATE = """Question: {question}

Given an input question, you will search the Knowledge Base on Bedrock User Guide to answer the user question. 
If the knowledge base search results does not return any answer you can try answering it to the best of your ability but do not answer anything you do not know. Do not hallucinate.
Using this knowledge base search results you will ALWAYS execute the appropriate action group API to measure and detect the hallucination on that knowledge base search results.

Remove any XML tags from the knowledge base search results and final user response.
"""

In [ ]:
%%time


agent_answers = list()
for index, row in questions_df.iterrows():
    session_id = str(uuid.uuid1())
    final_agent_answer = None
    question_id = row['question_id']
    question_text = row['question']
    gt_answer = row['ground_truth_answer']
    logger.info(f"-------------Question ID :: {question_id} Question_text :: {question_text} -------------------")
    final_agent_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                           USER_PROMPT_TEMPLATE.format(question=question_text),
                                           agent_id, 
                                           agent_alias_id, 
                                           session_id = session_id, 
                                           enable_trace = True,
                                           end_session = False,
                                           trace_filename_prefix = 'lab2_hallucination_agent_trace',
                                           turn_number = index)
    
    time.sleep(20) # to avoid throttling errors if any, you can update it to greater than 60, say 65 
    #print(f"final_agent_answer --> {final_agent_answer}")
    agent_answers.append(final_agent_answer)
    format_final_response(question_id = question_id, 
                          question = question_text, 
                          final_answer = final_agent_answer, 
                          lab_number=2, 
                          turn_number=index, 
                          show_detailed=True)



,Question ID,User Question,Agent/Chatbot Response,KB Response,Answer Score
0,Q001,What models are supported by bedrock agents?,"For question = What models are supported by bedrock agents? .. Getting a customer service agent to help you, please wait and stay connected ....","IMPORTANT: USE THIS RESULT VERBATIM. DO NOT SUMMARIZE: <answer_part><text>Amazon Bedrock supports the following models for use with agents:- Anthropic Claude 3 Opus- AI21 Labs Jurassic-2 Mid and Ultra- Cohere Command, Command Light, Command R, and Command R+- Meta Llama 2 Chat 13B and 70B- Meta Llama 2 13B and 70B- Meta Llama 3 8B Instruct, 70B Instruct, 3.1 8B Instruct, 3.1 70B Instruct, and 3.1 405B Instruct- Meta Llama 3.2 1B Instruct, 3B Instruct, 11B Instruct, and 90B Instruct- Mistral AI Mistral 7B Instruct, Mistral Large, Mistral Large 2 24.07, Mixtral 8X7B Instruct, and Mistral Small</text><sources><source>4</source></sources></answer_part><answer_part><text>The models support various features like knowledge base integration, fine-tuning, tool use, and the Converse API to different extents. For example, the Meta Llama 3.1 models support agents, fine-tuning, tool use, and the Converse API, while the Mistral AI models only support agents and the Converse API except Mistral Large 2 which does not support the Converse API.</text><sources><source>4</source></sources></answer_part>",0.460934


,Question ID,User Question,Agent/Chatbot Response,KB Response,Answer Score
0,Q002,Which models can I use with Amazon Bedrock Agents?,"For question = Which models can I use with Amazon Bedrock Agents? .. Getting a customer service agent to help you, please wait and stay connected ....",You can use the following models with Amazon Bedrock Agents:- Mistral AI Mistral Large 2 24.07- Meta Llama 3.1 Instruct- AI21 Jamba-Instruct- Claude 3.5 SonnetAmazon Bedrock supports using different foundation models from various providers. The supported models are listed on the Models supported page in the Amazon Bedrock documentation.,0.593483


,Question ID,User Question,Agent/Chatbot Response,KB Response,Answer Score
0,Q003,Which are the dates for reinvent 2024?,"For question = Which are the dates for reinvent 2024? .. Getting a customer service agent to help you, please wait and stay connected ....","Unfortunately, I could not find any information about the dates for AWS re:Invent 2024 in the provided search results. The search results appear to contain documentation and release notes for the Amazon Bedrock service, but do not mention the AWS re:Invent conference dates.",0.015347


,Question ID,User Question,Agent/Chatbot Response,KB Response,Answer Score
0,Q004,What is the Model ID of Amazon Titan Text Premier,The model ID for Amazon Titan Text G1 - Premier is amazon.titan-text-premier-v1:0.,The model ID for Amazon Titan Text G1 - Premier is amazon.titan-text-premier-v1:0.,0.947762


CPU times: user 149 ms, sys: 20 ms, total: 169 ms
Wall time: 7min


### <a name="7">Monitor the SNS messages received for Human in the Loop setup </a>
(<a href="#0">Go to top</a>)

- To verify the actual SNS message count, you can view the latest  Lambda cloud watch logs following the instructions as given in the [LINK](https://docs.aws.amazon.com/lambda/latest/dg/monitoring-cloudwatchlogs-view.html) . Search for the string `Received SNS message ::` inside the cloudwatch logs. The lambda function for this notebook is called `LambdaAgentsHallucinationDetection`

- To check the SNS message count, you can monitor the number of messages in the SNS topic `reinvent2024_hallucination_lab2b_topic` via cloudwatch metric `NumberOfMessagesPublished` as given in the [LINK](https://docs.aws.amazon.com/sns/latest/dg/sns-monitoring-using-cloudwatch.html)

### <a name="8">[Be Frugal] Clean up resources </a>
(<a href="#0">Go to top</a>)


##### In the following cell, we offer the option to raise an exception to avoid auto-executing the next block of lines and optionally clean up all resources. This is useful when the `Kernel > run all` option is used.

`Please be frugal if you choose to enable this exception in the code cell below. By default it is disabled and all resources will be cleaned up immediately to avoid additional costs.`

##### Within the same kernel session, this will allow experimentation with different prompts without having to recreate agent resources (takes ~5 minutes)

In [18]:
# this avoids auto-cleanup
raise Exception('Avoiding Auto-Cleanup of Amazon Bedrock Agent Resources')

Exception: Avoiding Auto-Cleanup of Amazon Bedrock Agent Resources

In [ ]:
%%time

cleanup_infrastructure(agent_action_group_response, 
                       lambda_name, 
                       lambda_function, 
                       lambda_role_name, 
                       agent_id, 
                       agent_alias_id, 
                       agent_role_name, 
                       bucket_name, 
                       schema_key, 
                       agent_bedrock_policy, 
                       agent_s3_schema_policy, 
                       agent_kb_schema_policy, 
                       kb_db_bedrock_policy, 
                       kb_db_aoss_policy, 
                       kb_db_s3_policy, 
                       kb_db_role_name, 
                       kb_db_collection_name, 
                       kb_db_opensearch_collection_response, 
                       knowledge_base_db_id, 
                       sns_topic_name)

---

### <a name="9">Challenge Exercise :: Try it Yourself! </a>
(<a href="#0">Go to top</a>)





<div style="border: 4px solid coral; text-align: left; margin: auto;">
    <br>
    <p style="text-align: center; margin: auto;"><b>Try the following exercises on this lab and note the observations.</b></p>
<p style=" text-align: left; margin: auto;">
<ol>
 <li>Try a new set of questions to test against the agent, reference the Amazon Bedrock User Guide to come up with these questions. </li>
<li> Notice the questions where the human in the loop are getting invoked? Does question reframing/rewriting help avoid it? </li>
<li> Try different chunking strategies supported by Bedrock Knowledge base and ask the same set of questions to compare and contrast against each chunking strategy for this use-case. </li>
<li> Try additional RAGAS metrics from the documentation, <a href="https://docs.ragas.io/en/v0.1.21/concepts/metrics/index.html">RAGAS METRICS</a> </li>
    <li> Try different open source PDF(s) to verify . </li>
</ol>
<br>
</p>
</div>



## Conclusion
We now have an understanding of how to detect, measure and remediate hallucinations with Human in the Loop even after applying RAG workflows with an agentic AI workflow. 
Furthermore, each failure scenario could be an opportunity to improve the raw datasource for better clarity.


### Take aways 
- Adapt this notebook to create newer hallucination detection and thresholding mechanisms to involve human in the loop for your use-case.

## Thank You